In [50]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 20: Trench Map ---</h2><p>With the scanners fully deployed, you turn their attention to mapping the floor of the ocean trench.</p>
<p>When you get back the image from the scanners, it seems to just be random noise. Perhaps you can combine an image enhancement algorithm and the input image (your puzzle input) to clean it up a little.</p>
<p>For example:</p>
<pre><code>..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###
</code></pre>

<p>The first section is the <em>image enhancement algorithm</em>. It is normally given on a single line, but it has been wrapped to multiple lines in this example for legibility. The second section is the <em>input image</em>, a two-dimensional grid of <em>light pixels</em> (<code>#</code>) and <em>dark pixels</em> (<code>.</code>).</p>
<p>The image enhancement algorithm describes how to enhance an image by <em>simultaneously</em> converting all pixels in the input image into an output image. Each pixel of the output image is determined by looking at a 3x3 square of pixels centered on the corresponding input image pixel. So, to determine the value of the pixel at (5,10) in the output image, nine pixels from the input image need to be considered: (4,9), (4,10), (4,11), (5,9), (5,10), (5,11), (6,9), (6,10), and (6,11). These nine input pixels are combined into a single binary number that is used as an index in the <em>image enhancement algorithm</em> string.</p>
<p>For example, to determine the output pixel that corresponds to the very middle pixel of the input image, the nine pixels marked by <code>[...]</code> would need to be considered:</p>
<pre><code># . . # .
#[. . .].
#[# . .]#
.[. # .].
. . # # #
</code></pre>
<p>Starting from the top-left and reading across each row, these pixels are <code>...</code>, then <code>#..</code>, then <code>.#.</code>; combining these forms <code>...#...#.</code>. By turning dark pixels (<code>.</code>) into <code>0</code> and light pixels (<code>#</code>) into <code>1</code>, the binary number <code>000100010</code> can be formed, which is <code>34</code> in decimal.</p>
<p>The image enhancement algorithm string is exactly 512 characters long, enough to match every possible 9-bit binary number. The first few characters of the string (numbered starting from zero) are as follows:</p>
<pre><code>0         10        20        30  <em>34</em>    40        50        60        70
|         |         |         |   <em>|</em>     |         |         |         |
..#.#..#####.#.#.#.###.##.....###.<em>#</em>#.#..###.####..#####..#....#..#..##..##
</code></pre>
<p>In the middle of this first group of characters, the character at index 34 can be found: <code>#</code>. So, the output pixel in the center of the output image should be <code>#</code>, a <em>light pixel</em>.</p>
<p>This process can then be repeated to calculate every pixel of the output image.</p>
<p>Through advances in imaging technology, the images being operated on here are <em>infinite</em> in size. <em>Every</em> pixel of the infinite output image needs to be calculated exactly based on the relevant pixels of the input image. The small input image you have is only a small region of the actual infinite input image; the rest of the input image consists of dark pixels (<code>.</code>). For the purposes of the example, to save on space, only a portion of the infinite-sized input and output images will be shown.</p>
<p>The starting input image, therefore, looks something like this, with more dark pixels (<code>.</code>) extending forever in every direction not shown here:</p>
<pre><code>...............
...............
...............
...............
...............
.....#..#......
.....#.........
.....##..#.....
.......#.......
.......###.....
...............
...............
...............
...............
...............
</code></pre>
<p>By applying the image enhancement algorithm to every pixel simultaneously, the following output image can be obtained:</p>
<pre><code>...............
...............
...............
...............
.....##.##.....
....#..#.#.....
....##.#..#....
....####..#....
.....#..##.....
......##..#....
.......#.#.....
...............
...............
...............
...............
</code></pre>
<p>Through further advances in imaging technology, the above output image can also be used as an input image! This allows it to be enhanced <em>a second time</em>:</p>
<pre><code>...............
...............
...............
..........#....
....#..#.#.....
...#.#...###...
...#...##.#....
...#.....#.#...
....#.#####....
.....#.#####...
......##.##....
.......###.....
...............
...............
...............
</code></pre>
<p>Truly incredible - now the small details are really starting to come through. After enhancing the original input image twice, <code><em>35</em></code> pixels are lit.</p>
<p>Start with the original input image and apply the image enhancement algorithm twice, being careful to account for the infinite size of the images. <em>How many pixels are lit in the resulting image?</em></p>
</article>


In [51]:
from pprint import pprint


example = """
..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###
"""


class Image:
    def __init__(self, s: str | None = None) -> None:
        if s is not None:
            alg, img = re.split(r"\n\s*\n", s.strip())
            self.alg = "".join(alg.splitlines())
            self.img = [list(l) for l in img.splitlines()]
            self.off_image = "."
        else:
            self.alg = None
            self.img = None

    def enhance(self) -> Image:
        rows, cols = len(self.img), len(self.img[0])
        off_image = self.alg[int(self.off_image == "#") * 9]

        temp_image = [
            [self.off_image] * (cols + 4),
            [self.off_image] * (cols + 4),
            *[
                [self.off_image, self.off_image]
                + line
                + [self.off_image, self.off_image]
                for line in self.img
            ],
            [self.off_image] * (cols + 4),
            [self.off_image] * (cols + 4),
        ]

        new_image = [[off_image] * (cols + 4) for _ in range(rows + 4)]
        rows1, cols1 = cols + 4, rows + 4

        for r, c in product(range(1, rows1 - 1), range(1, cols1 - 1)):
            number = 0

            for dr, dc in product(range(-1, 2), repeat=2):
                number = 2 * number + int(temp_image[r + dr][c + dc] == "#")

            new_image[r][c] = self.alg[number]

        image = Image()
        image.alg = self.alg
        image.img = new_image
        image.off_image = off_image

        return image

    def lights(self) -> int:
        return sum(
            1
            for r, c in product(range(len(self.img)), range(len(self.img[0])))
            if self.img[r][c] == "#"
        )

    def __repr__(self):
        return "\n".join("".join(l) for l in self.img)

    @classmethod
    def part1(cls, s: str) -> int:
        return Image(s).enhance().enhance().lights()

    @classmethod
    def part2(cls, s: str) -> int:
        current = Image(s)
        for _ in range(50):
            current = current.enhance()
        return current.lights()


assert Image.part1(example) == 35

In [52]:
with open("../input/day20.txt") as f:
    puzzle = f.read()


print(f"Part I: { Image.part1(puzzle) }")

Part I: 5359


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>5359</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You still can't quite make out the details in the image. Maybe you just didn't <a href="https://en.wikipedia.org/wiki/Kernel_(image_processing)" target="_blank">enhance</a> it <span title="Yeah, that's definitely the problem.">enough</span>.</p>
<p>If you enhance the starting input image in the above example a total of <em>50</em> times, <code><em>3351</em></code> pixels are lit in the final output image.</p>
<p>Start again with the original input image and apply the image enhancement algorithm 50 times. <em>How many pixels are lit in the resulting image?</em></p>
</article>

</main>


In [53]:
assert Image.part2(example) == 3351

In [54]:
print(f"Part II: { Image.part2(puzzle) }")

Part II: 12333


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>12333</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
